![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForMultipleChoice.ipynb)

## Import ONNX DistilBertForMultipleChoice models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForMultipleChoice` is only available since in `Spark NLP 5.6.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for question answering via `DistilBertForMultipleChoice` or `TFDistilBertForMultipleChoice`. These models are usually under `Multiple Choice` category and have `bert` in their labels
- Reference: [DistilBertForMultipleChoice](https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForMultipleChoice)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.2 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [distilbert-base-cased](https://huggingface.co/distilbert-base-cased) model from HuggingFace as an example and load it as a `ORTModelForMultipleChoice`, representing an ONNX model.
- In addition to the DistilBERT model, we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from transformers import DistilBertTokenizer
from optimum.onnxruntime import ORTModelForMultipleChoice

MODEL_NAME = "Gladiator/distilbert-base-uncased_swag_mqa"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForMultipleChoice.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

('onnx_models/Gladiator/distilbert-base-uncased_swag_mqa/tokenizer_config.json',
 'onnx_models/Gladiator/distilbert-base-uncased_swag_mqa/special_tokens_map.json',
 'onnx_models/Gladiator/distilbert-base-uncased_swag_mqa/vocab.txt',
 'onnx_models/Gladiator/distilbert-base-uncased_swag_mqa/added_tokens.json')

We are missing the `vocab.txt` from the tokenizer. Let's save the tokenizer and move this file to `assets` folder which Spark NLP will look for

In [3]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [4]:
!ls -lR {EXPORT_PATH}

onnx_models/Gladiator/distilbert-base-uncased_swag_mqa:
total 261700
drwxr-xr-x 2 root root      4096 Jun 25 10:36 assets
-rw-r--r-- 1 root root       577 Jun 25 10:36 config.json
-rw-r--r-- 1 root root 267957763 Jun 25 10:36 model.onnx
-rw-r--r-- 1 root root       695 Jun 25 10:36 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 25 10:36 tokenizer_config.json

onnx_models/Gladiator/distilbert-base-uncased_swag_mqa/assets:
total 228
-rw-r--r-- 1 root root 231508 Jun 25 10:36 vocab.txt


Voila! We have our vocab.txt inside assets directory

## Import and Save DistilBertForMultipleChoice in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [5]:
!pip install -q spark-nlp==6.0.3 pyspark==3.5.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.0/713.0 kB 15.0 MB/s eta 0:00:00


In [6]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  6.0.3
Apache Spark version:  3.5.1


- Let's use `loadSavedModel` functon in `DistilBertForMultipleChoice` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForMultipleChoice` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [7]:
from sparknlp.annotator import DistilBertForMultipleChoice

distilBertMultpleChoiceClassifier = DistilBertForMultipleChoice.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question", "document_context"])\
  .setOutputCol("answer")

Let's save it on disk so it is easier to be moved around and also be used later via .load function

In [8]:
distilBertMultpleChoiceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [9]:
!rm -rf {EXPORT_PATH}

Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny `DistilBertForMultipleChoice` model in Spark NLP 🚀 pipeline!

In [10]:
data = spark.createDataFrame([
    ("In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.",
     "It is eaten with a fork and a knife, It is eaten while held in the hand."),
    ("The Eiffel Tower is located in which country?", "Germany, France, Italy"),
    ("Which animal is known as the king of the jungle?", "Lion, Elephant, Tiger, Leopard"),
    ("Water boils at what temperature?", "90°C, 120°C, 100°C"),
    ("Which planet is known as the Red Planet?", "Jupiter, Mars, Venus"),
    ("Which language is primarily spoken in Brazil?", "Spanish, Portuguese, English"),
    ("The Great Wall of China was built to protect against invasions from which group?",
     "The Greeks, The Romans, The Mongols, The Persians"),
    ("Which chemical element has the symbol 'O'?", "Oxygenm, Osmium, Ozone"),
    ("Which continent is the Sahara Desert located in?", "Asia, Africa, South America"),
    ("Which artist painted the Mona Lisa?", "Vincent van Gogh, Leonardo da Vinci, Pablo Picasso")
], ["question", "choices"])

data.show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|question                                                                                  |choices                                                                 |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |
|Which animal is known as the king of the jungle?                                          |Lion, Elephant, Tiger, Leopard                                          |
|Wat

In [11]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import DistilBertForMultipleChoice
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "choices"]) \
    .setOutputCols(["document_question", "document_choices"])

distilbert_model = DistilBertForMultipleChoice() \
    .load(f"./{MODEL_NAME}_spark_nlp_onnx") \
    .setInputCols(["document_question", "document_choices"]) \
    .setOutputCol("answer") \
    .setBatchSize(4)

pipeline = Pipeline(stages=[
    document_assembler,
    distilbert_model
])

model = pipeline.fit(data)
results = model.transform(data)

results.selectExpr("question", "choices", "answer.result[0] as predicted_answer").show(truncate=False)

+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|question                                                                                  |choices                                                                 |predicted_answer                   |
+------------------------------------------------------------------------------------------+------------------------------------------------------------------------+-----------------------------------+
|In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced.|It is eaten with a fork and a knife, It is eaten while held in the hand.|It is eaten with a fork and a knife|
|The Eiffel Tower is located in which country?                                             |Germany, France, Italy                                                  |Germany                    